# Xbox Datenaufbereitung und Lineare Regression

In diesem Notebook werden die Xbox-Daten verarbeitet und ein Modell für die Vorhersage von Konsolenpreisen mithilfe von Linearer Regression erstellt.

## 1. Daten laden und anzeigen
Hier werden die Daten aus der CSV-Datei geladen und die ersten Zeilen angezeigt.

In [1]:

import os
import pandas as pd

# Datei mit den Daten
data_file = "/Users/Hannes/Samsaru/asgoodasnew_products.csv"

# Daten laden
if os.path.exists(data_file):
    df = pd.read_csv(data_file)
    display(df.head())
else:
    print(f"Die Datei {data_file} existiert nicht!")


,scrape_date,title,price,brand,category,variant
0,2025-01-09 10:49:40,Microsoft Xbox Series X - 1TB schwarz,589.00,Microsoft,Konsolen,neu
1,2025-01-09 10:49:40,Microsoft Xbox Series X - 1TB schwarz,569.00,Microsoft,Konsolen,wie neu
2,2025-01-09 10:49:40,Microsoft Xbox Series X - 1TB schwarz,549.00,Microsoft,Konsolen,sehr gut
3,2025-01-09 10:49:40,Microsoft Xbox Series X - 1TB schwarz,539.00,Microsoft,Konsolen,gut
4,2025-01-09 10:49:40,Microsoft Xbox Series S - 1TB carbon black,443.72,Microsoft,Konsolen,neu


## 2. Datenverarbeitung
Hier wird der Titel in `model` und `storage_capacity` aufgeteilt. Außerdem werden neue Features wie `variant_score` und `price_deviation` berechnet.

In [2]:

# Preis in numerische Werte umwandeln
df['price'] = pd.to_numeric(df['price'], errors='coerce')
df.dropna(subset=['price'], inplace=True)

# Titel aufteilen in Modell und Speicherkapazität
df[['model', 'storage_capacity']] = df['title'].str.extract(r'(Microsoft Xbox [^–]+) - ([^ ]+)')
df.drop(columns=['title'], inplace=True)

# Zustand in numerische Werte kodieren
df['variant_score'] = df['variant'].map({'neu': 1, 'wie neu': 0.8, 'sehr gut': 0.6, 'gut': 0.4})

# Durchschnittspreis nach Modell berechnen
df['avg_price_by_title'] = df.groupby('model')['price'].transform('mean')
df['price_deviation'] = df['price'] - df['avg_price_by_title']

# Marke und Kategorie kodieren
df['brand'] = df['brand'].astype('category').cat.codes
df['category'] = df['category'].astype('category').cat.codes

# Speichergröße in numerische Werte umwandeln
def parse_storage_capacity(value):
    if 'TB' in value:
        return int(float(value.replace('TB', '').strip()) * 1024)  # TB zu GB umwandeln
    elif 'GB' in value:
        return int(value.replace('GB', '').strip())
    return None

df['storage_capacity'] = df['storage_capacity'].apply(parse_storage_capacity)

# Entferne Zeilen mit ungültiger Speichergröße
df.dropna(subset=['storage_capacity'], inplace=True)

# Verarbeitete Daten anzeigen
display(df)



,scrape_date,price,brand,category,variant,model,storage_capacity,variant_score,avg_price_by_title,price_deviation
0,2025-01-09 10:49:40,589.00,0,0,neu,Microsoft Xbox Series X,1024,1.0,492.3915,96.6085
1,2025-01-09 10:49:40,569.00,0,0,wie neu,Microsoft Xbox Series X,1024,0.8,492.3915,76.6085
2,2025-01-09 10:49:40,549.00,0,0,sehr gut,Microsoft Xbox Series X,1024,0.6,492.3915,56.6085
3,2025-01-09 10:49:40,539.00,0,0,gut,Microsoft Xbox Series X,1024,0.4,492.3915,46.6085
4,2025-01-09 10:49:40,443.72,0,0,neu,Microsoft Xbox Series S,1024,1.0,343.5250,100.1950
5,2025-01-09 10:49:40,481.99,0,0,sehr gut,Microsoft Xbox Series X,1024,0.6,492.3915,-10.4015
6,2025-01-09 10:49:40,321.47,0,1,neu,Microsoft Xbox One S,500,1.0,236.6450,84.8250
7,2025-01-09 10:49:40,399.91,0,0,wie neu,Microsoft Xbox Series S,1024,0.8,343.5250,56.3850
8,2025-01-09 10:49:40,202.17,0,0,gut,Microsoft Xbox Series S,512,0.4,343.5250,-141.3550
9,2025-01-09 10:49:40,401.72,0,0,sehr gut,Microsoft Xbox Series X,1024,0.6,492.3915,-90.6715


## 3. Lineare Regression
Hier wird ein Modell zur Vorhersage der Konsolenpreise mit Linearer Regression trainiert.

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score

# Feature-Auswahl
features = ['storage_capacity', 'variant_score', 'price_deviation', 'brand', 'category']
X = df[features]
y = df['price']

# Daten aufteilen in Training und Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modell initialisieren und trainieren (mit Regularisierung)
ridge_model = Ridge(alpha=3.0)  # alpha ist der Regularisierungsparameter (kann angepasst werden)
ridge_model.fit(X_train, y_train)

# Vorhersagen
y_pred = ridge_model.predict(X_test)

# Ergebnisse bewerten
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

# Ergebnisse ausgeben
print(f"R²-Wert (Ridge Regression): {r2:.2f}")
print(f"Mean Squared Error (Ridge Regression): {mse:.2f}")

# Beispiel: Zeige die Feature-Namen und R²-Wert
if features:
    print("Verwendete Features:", features)


R²-Wert (Ridge Regression): 0.78
Mean Squared Error (Ridge Regression): 2386.21
Verwendete Features: ['storage_capacity', 'variant_score', 'price_deviation', 'brand', 'category']


## 4. Speichern des Modells
Das trainierte Modell wird gespeichert, damit es später wiederverwendet werden kann.

In [4]:

import pickle

model_file = "linear_model.pkl"

# Modell speichern
try:
    with open(model_file, 'wb') as f:
        pickle.dump({"model": model, "features": features, "r2": r2}, f)
    print(f"Modell erfolgreich in {os.path.abspath(model_file)} gespeichert.")
except Exception as e:
    print(f"Fehler beim Speichern des Modells: {e}")


Fehler beim Speichern des Modells: name 'model' is not defined


In [5]:

import gradio as gr

# Funktion zur dynamischen Filterung der Dropdown-Optionen
def get_dynamic_dropdowns(console_model=None):
    filtered_df = df.copy()

    # Wenn ein Konsolenmodell ausgewählt ist, filtere basierend darauf
    if console_model:
        filtered_df = filtered_df[filtered_df['model'] == console_model]

    # Optionen für Speicherkapazität und Zustand basierend auf der Auswahl
    storage_capacity_options = filtered_df['storage_capacity'].unique().astype(str).tolist()
    condition_options = filtered_df['variant'].unique().tolist()

    return storage_capacity_options, condition_options

# Funktion zur Vorhersage
def predict_price(console_model, storage_capacity, condition):
    try:
        # Filtere die Eingaben und bereite den Feature-Vektor vor
        filtered_df = df[
            (df['model'] == console_model) &
            (df['storage_capacity'] == int(storage_capacity)) &
            (df['variant'] == condition)
        ]

        if filtered_df.empty:
            return "Keine Daten verfügbar", None

        # Durchschnittspreis berechnen (oder andere Berechnung)
        predicted_price = filtered_df['price'].mean()

        return round(predicted_price, 2), r2  # Rückgabe von Preis und R²-Wert
    except Exception as e:
        return f"Fehler: {str(e)}", None

# Dropdown-Optionen vorbereiten
console_model_options = df['model'].unique().tolist()

# Gradio-Interface erstellen
with gr.Blocks() as interface:
    gr.Markdown("# **Samsaru-Verkaufspreisvorhersage**")
    gr.Markdown("Finden Sie Verkaufspreise basierend auf aktuellen Daten.")

    with gr.Row():
        with gr.Column():
            console_model = gr.Dropdown(
                label="Modell",
                choices=console_model_options,
                value=console_model_options[0]  # Standardwert
            )
            storage_capacity = gr.Dropdown(
                label="Speicherkapazität",
                choices=[],
                value=None,
                interactive=True  # Ermöglicht das dynamische Aktualisieren
            )
            condition = gr.Dropdown(
                label="Zustand",
                choices=[],
                value=None,
                interactive=True  # Ermöglicht das dynamische Aktualisieren
            )
            submit_button = gr.Button("Vorhersage starten")

        with gr.Column():
            predicted_price = gr.Textbox(label="Vorhergesagter Marktpreis")
            r2_value = gr.Textbox(label="R²-Wert des Modells")

    # Dynamische Aktualisierung der Dropdown-Optionen
    def update_dropdowns(console_model):
        storage_options, condition_options = get_dynamic_dropdowns(console_model)
        return gr.update(choices=storage_options, value=storage_options[0] if storage_options else None), \
               gr.update(choices=condition_options, value=condition_options[0] if condition_options else None)

    console_model.change(
        update_dropdowns,
        inputs=[console_model],
        outputs=[storage_capacity, condition]
    )

    # Verbinde die Eingaben und Ausgaben mit der Vorhersagefunktion
    submit_button.click(
        predict_price,
        inputs=[console_model, storage_capacity, condition],
        outputs=[predicted_price, r2_value]
    )

# Interface starten
interface.launch()



/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


## 5. Fazit
Das Modell wurde erfolgreich trainiert und gespeichert. Es kann verwendet werden, um Konsolenpreise basierend auf ihren Merkmalen vorherzusagen.